In [12]:
import unittest
import yfinance as yf
import talib
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

# Descargar los datos históricos de Bitcoin
btc = yf.download('BTC-USD', period="1y", interval="1d")

# Agregar las bandas de Bollinger y el indicador MACD
window = 20
no_of_std = 2

# Agregar la media móvil exponencial de 20 días
btc['MA20'] = talib.EMA(btc['Close'], timeperiod=window)

# Agregar las bandas de Bollinger
btc['Upper_Band'], btc['MA20'], btc['Lower_Band'] = talib.BBANDS(btc['Close'], timeperiod=window, nbdevup=no_of_std, nbdevdn=no_of_std, matype=0)

# Agregar el indicador MACD
btc['MACD'], btc['Signal Line'], _ = talib.MACD(btc['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Agregar las señales de compra y venta
btc['Buy'] = np.where((btc['Close'] > btc['Upper_Band']) & (btc['MACD'] > btc['Signal Line']), 1, 0)
btc['Sell'] = np.where((btc['Close'] < btc['Lower_Band']) & (btc['MACD'] < btc['Signal Line']), 1, 0)

# Agregar la evolución del portfolio
btc['Shares'] = np.where(btc['Buy'] == 1, 1000 / btc['Close'], 0)
btc['Shares'] = np.where(btc['Sell'] == 1, -btc['Shares'], btc['Shares'])
btc['Shares'] = btc['Shares'].cumsum()
btc['Portfolio Value'] = btc['Shares'] * btc['Close']

# Mostrar la evolución del valor del portfolio
fig = px.line(btc, x=btc.index, y='Portfolio Value', title='Evolución del Valor del Portfolio en USD')
fig.update_xaxes(title_text='Fecha')
fig.update_yaxes(title_text='Valor')
fig.show()

# Mostrar la evolución del precio de cierre de Bitcoin
fig = px.line(btc, x=btc.index, y='Close', title='Evolución del Precio de Cierre de Bitcoin en USD')
fig.update_xaxes(title_text='Fecha')
fig.update_yaxes(title_text='Precio')
fig.show()

# Mostrar las señales de compra y venta en el precio de cierre de Bitcoin
fig = px.scatter(btc, x=btc.index, y='Close', color='Buy', symbol='Buy', title='Señales de Compra y Venta en el Precio de Cierre de Bitcoin')
fig.add_scatter(x=btc.loc[btc['Sell'] == 1].index, y=btc.loc[btc['Sell'] == 1]['Close'], name='Venta', mode='markers', marker=dict(color='red', symbol='cross'))
fig.update_xaxes(title_text='Fecha')
fig.update_yaxes(title_text='Precio')
fig.show()

# Mostrar los resultados
print(btc['Portfolio Value'].tail())

#Agregamos pruebas unitarias para ver si el codigo sirve
class TestStrategy(unittest.TestCase):
    def setUp(self):
        self.btc = yf.download('BTC-USD', period="1mo", interval="5m")
        self.window = 20
        self.no_of_std = 2
        
    def test_sma(self): #test_sma: prueba que el promedio móvil simple (SMA) del precio de cierre de los datos de Bitcoin-USD tiene la misma longitud que los datos del precio de cierre.
        self.btc['MA20'] = talib.EMA(self.btc['Close'], timeperiod=self.window)
        self.assertEqual(len(self.btc['MA20']), len(self.btc['Close']))
        
    def test_bollinger_bands(self): #test_bollinger_bands comprueba que las Bandas de Bollinger superior e inferior, y la SMA del precio de cierre, tienen la misma longitud que los datos del precio de cierre.
        self.btc['Upper_Band'], self.btc['MA20'], self.btc['Lower_Band'] = talib.BBANDS(self.btc['Close'], timeperiod=self.window, nbdevup=self.no_of_std, nbdevdn=self.no_of_std, matype=0)
        self.assertEqual(len(self.btc['Upper_Band']), len(self.btc['Close']))
        self.assertEqual(len(self.btc['MA20']), len(self.btc['Close']))
        self.assertEqual(len(self.btc['Lower_Band']), len(self.btc['Close']))
        
    def test_macd(self): #test_macd prueba que la divergencia de convergencia del promedio móvil (MACD) y la línea de señal tienen la misma longitud que los datos del precio de cierre.
        self.btc['MACD'], self.btc['Signal Line'], _ = talib.MACD(self.btc['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        self.assertEqual(len(self.btc['MACD']), len(self.btc['Close']))
        self.assertEqual(len(self.btc['Signal Line']), len(self.btc['Close']))
        
    def test_buy_sell_signals(self):
        self.btc['Buy'] = np.where((self.btc['Close'] > self.btc['Upper_Band']) & (self.btc['MACD'] > self.btc['Signal Line']), 1, 0)
        self.btc['Sell'] = np.where((self.btc['Close'] < self.btc['Lower_Band']) & (self.btc['MACD'] < self.btc['Signal Line']), 1, 0)
        self.assertEqual(len(self.btc['Buy']), len(self.btc['Close']))
        self.assertEqual(len(self.btc['Sell']), len(self.btc['Close']))
        
    def test_portfolio_evolution(self):
        self.btc['Shares'] = np.where(self.btc['Buy'] == 1, 1000 / self.btc['Close'], 0)
        self.btc['Shares'] = np.where(self.btc['Sell'] == 1, -self.btc['Close'],self.btc['Close'])
        self.btc['Shares'] = self.btc['Shares'].cumsum()
        self.btc['Portfolio Value'] = self.btc['Shares'] * self.btc['Close']


[*********************100%***********************]  1 of 1 completed


Date
2023-01-31 00:00:00+00:00    27612.453680
2023-02-01 00:00:00+00:00    28309.929981
2023-02-02 00:00:00+00:00    28009.335798
2023-02-03 00:00:00+00:00    27982.427943
2023-02-04 00:00:00+00:00    27926.687081
Name: Portfolio Value, dtype: float64
